In [146]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import scipy

import warnings
warnings.filterwarnings('ignore')
 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder
from sklearn.cross_validation import train_test_split 
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,f1_score
from skmultilearn.problem_transform import LabelPowerset
from xgboost import XGBClassifier
from sklearn import pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

import commonfunction


model = trainmodel()
model.transform_target()
model.prepare_data()

train_df  = model.get_traindata()
target = model.get_target()

X_train, X_test, Y_train, Y_test = train_test_split(train_df,target, test_size=0.33, random_state=42) 

In [147]:
pipeline  = Pipeline([
    ('select_column', ColumnSelector(key='content')),
    ('cvec',CountVectorizer(ngram_range=(1,3))),
    ('svd', TruncatedSVD(n_components=1000)),
    ('classifier', LabelPowerset())      
])
models1 = {     
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'XGBClassifier' : XGBClassifier(),
    'LogisticRegression' : LogisticRegression()
    'MultinomialNB' : MultinomialNB()
    #'SVC': SVC()
}

params1 = {          
    'RandomForestClassifier': {'classifier__classifier__max_depth': [16, 32]},
    'AdaBoostClassifier' :  {'classifier__classifier__n_estimators': [16, 32]}
    'GradientBoostingClassifier': { 'classifier__classifier__n_estimators': [16, 32], 
                                   'classifier__classifier__learning_rate': [0.8, 1.0] 
                                  }
    'XGBClassifier' : { 'classifier__classifier__max_depth' : [6],
                      'classifier__classifier__learning_rate': [0.05,0.001],
                      'classifier__classifier__max_depth': [6],
                      'classifier__classifier__min_child_weight': [11],              
                      'classifier__classifier__subsample': [0.8],
                      'classifier__classifier__colsample_bytree': [0.7],
                      'classifier__classifier__n_estimators': [100]                                
    }
    
    'LogisticRegression' : {
        'C': [0.001, 0.01, 0.1, 1, 10]
    }
    
}

 

helper1 = EstimatorSelectionHelper(pipeline, models1, params1)
helper1.fit(X_train, Y_train, scoring='f1_weighted', n_jobs=2)

helper1.score_summary()

Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/jayap/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jayap/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:  4.3min finished


AdaBoostClassifier
RandomForestClassifier


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:  4.2min finished
/home/jayap/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:61: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,estimator,min_score,mean_score,max_score,std_score,classifier__classifier__max_depth,classifier__classifier__n_estimators
3,RandomForestClassifier,0.588073,0.595691,0.604637,0.00682695,32,NaN
2,RandomForestClassifier,0.574378,0.590844,0.613742,0.016701,16,NaN
0,AdaBoostClassifier,0.191262,0.295278,0.415199,0.0921109,NaN,16
1,AdaBoostClassifier,0.186697,0.293286,0.413788,0.0932303,NaN,32


In [91]:
# test the classifier
prediction = helper1.predict(X_test)

print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))

AttributeError: EstimatorSelectionHelper instance has no attribute 'predict'

In [50]:
pipeline2 = Pipeline([
    ('select_column', ColumnSelector(key='content')),
    ('cvec',CountVectorizer(ngram_range=(1,3))),
    ( 'svd', TruncatedSVD() ),
    ('classifier',LabelPowerset()),        
])

In [ ]:
model = trainmodel()
model.transform_target()
 
model.prepare_data()


#Read the data for which the label to be predicted......
model.prepare_testdata("../../data/processed/test.csv")

#test_result is the predicted label 
test_result = model.predict_testdataset()

#Convert the binary n-array to the corresponding label
#resultant array will have two columns - cat, subcat

data = np.zeros(shape=[test_result.shape[0],2])
data_str = data.astype(str)

cx = scipy.sparse.coo_matrix(test_result)
for i,j,v in zip(cx.row, cx.col, cx.data):      
    if(data_str[i][0] == '0.0'):
        #print("Inside if stmt ...... ",i,j)
        data_str[i][0] = commonfunction.getlabelname(j)
    else:
        #print("Inside if else ...... ",i,j)
        data_str[i][1] = commonfunction.getlabelname(j)         

test_df = pd.read_csv("../../data/processed/test.csv")
test_id = test_df['document_id']
label_df = pd.DataFrame({' ':test_id,'cat':data_str[:,0],'subcat':data_str[:,1]})
label_df.to_csv("../../data/processed/predicted.csv")

In [30]:
# train the classifier
model = pipe.fit(X_train.T, Y_train)

# test the classifier
prediction = model.predict(X_test)

print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))

Hamming loss is 0.0611672278339
validation dataset accuracy is 0.265512265512
